In [56]:
from pymongo import MongoClient 
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


In [57]:
client =  MongoClient(os.getenv('MONGODB_URI'))
db = client['cosmetic']
collection = db['products']

In [58]:
products = collection.find({}).to_list()

In [59]:
df = pd.DataFrame(products)

<h2>Logistic input: text - output: category</h2>

In [60]:
X = df["name"].fillna("") + " " + df["description_raw"].fillna("")
y = df['category']

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [62]:
X_train.shape, y_train.shape

((720,), (720,))

In [63]:
X_test.shape, y_test.shape

((180,), (180,))

In [64]:
clf = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2))),  # text column
    ("logistic", LogisticRegression())
])

clf.fit(X_train, y_train)

,steps,"[('tfidf', ...), ('logistic', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [65]:
y_pred_lr = clf.predict(X_test)
print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr,zero_division=0))

=== Logistic Regression ===
Accuracy: 0.9055555555555556
                          precision    recall  f1-score   support

      Bộ Chăm Sóc Da Mặt       0.00      0.00      0.00         4
       Chống Nắng Da Mặt       1.00      1.00      1.00        15
          Clearance Sale       1.00      1.00      1.00         3
          Hỗ Trợ Trị Mụn       1.00      0.93      0.96        14
   Kem / Gel / Dầu Dưỡng       0.86      1.00      0.92        24
      Lotion / Sữa Dưỡng       0.83      1.00      0.91        10
           Mini / Sample       0.00      0.00      0.00         1
             Mặt Nạ Giấy       0.78      1.00      0.88         7
              Mặt Nạ Môi       1.00      0.50      0.67         2
              Mặt Nạ Mắt       0.00      0.00      0.00         3
              Mặt Nạ Ngủ       0.00      0.00      0.00         1
              Mặt Nạ Rửa       0.00      0.00      0.00         1
   Serum / Kem Dưỡng Mắt       0.50      0.50      0.50         4
       Serum / Tin

<h2>SVM for text --> skin type</h2>

In [66]:
from underthesea import word_tokenize
from gensim.models import FastText


In [67]:

def tokenize_vi(text):
    return word_tokenize(text, format="text").split()


In [68]:
df_skin  = df[(df['skin_type'] != '') & (df['skin_type'].isnull() == False)].reset_index(drop=True)
df_skin.head()

,_id,url,brand,category,description_raw,images,ingredient_raw,made_from,name,price,rating,skin_type,usage_tip,volume
0,691fdf445bc8d2cb663e1d9f,https://hasaki.vn/san-pham/dau-tay-trang-cocoo...,Cocoon,Tẩy Trang Mặt,Dầu Tẩy Trang Cocoon Chiết Xuất Hoa Hồng là dò...,[https://media.hcdn.vn/catalog/product/p/r/pro...,Thành phần chính: Dầu hoa hồng Damask: giúp d...,Vietnam,Dầu Tẩy Trang Cocoon Chiết Xuất Hoa Hồng 140ml,128000,4.9,Da thường/Mọi loại da,Dùng tay khô thoa sản phẩm lên da và mát-xa nh...,140ml
1,691fdf455bc8d2cb663e1da0,https://hasaki.vn/san-pham/nuoc-tay-trang-evel...,Eveline,Tẩy Trang Mặt,Nước Tẩy Trang Eveline Hyaluron Clinic B5 Cấp ...,[https://media.hcdn.vn/catalog/product/f/a/fac...,,Poland,Nước Tẩy Trang Eveline Hyaluron Clinic B5 Cấp ...,143000,5.0,Da thường/Mọi loại da,Đổ một lượng nước tẩy trang ra bông tẩy trang ...,500ml
2,691fdf465bc8d2cb663e1da1,https://hasaki.vn/san-pham/nuoc-tay-trang-coco...,Cocoon,Tẩy Trang Mặt,Nước Tẩy Trang Cocoon Hoa Hồng Cấp Ẩm Da là sả...,[https://media.hcdn.vn/catalog/product/p/r/pro...,Thành phần chính: Nước cất hoa hồng hữu cơ (o...,Vietnam,Nước Tẩy Trang Cocoon Hoa Hồng Cấp Ẩm Da 310ml,176000,0.0,Da khô/Hỗn hợp khô,"Lắc đều. Thấm sản phẩm lên bông tẩy trang, nh...",310ml
3,691fdf475bc8d2cb663e1da2,https://hasaki.vn/san-pham/dau-tay-trang-hada-...,Hada Labo,Tẩy Trang Mặt,Dầu Tẩy Trang Hada Labo Advanced Nourish Hyalu...,[https://media.hcdn.vn/catalog/product/f/a/fac...,Thành phần chính: Dầu Ô liu và Jojoba tự nhiê...,Vietnam,Dầu Tẩy Trang Hada Labo Sạch Sâu Dưỡng Ẩm Tối ...,168000,4.9,Da thường/Mọi loại da,"Bước 1: Giữ tay và mặt khô, lấy một lượng dầu ...",200ml
4,691fdf585bc8d2cb663e1da3,https://hasaki.vn/san-pham/nuoc-tay-trang-neut...,Neutrogena,Tẩy Trang Mặt,Nước Tẩy Trang Neutrogena Thanh Lọc Và Làm Sạc...,[https://media.hcdn.vn/catalog/product/p/r/pro...,"Water, PEG-6 Caprylic/Capric Glycerides, Polys...",Thailand,Nước Tẩy Trang Neutrogena Thanh Lọc Và Làm Sạc...,118000,4.8,Da thường/Mọi loại da,Bước 1: Làm ướt bông với nước tẩy trang. Bước...,400 ml


In [82]:
X_skin =  df_skin["description_raw"].fillna("") + df_skin['ingredient_raw'].fillna("") 
X_skin = X_skin.apply(lambda x: word_tokenize(x))

In [83]:
df_skin['skin_type'].value_counts()

skin_type
Da thường/Mọi loại da    422
Da mụn                   122
Da dầu/Hỗn hợp dầu       120
Da nhạy cảm              103
Da khô/Hỗn hợp khô        86
Name: count, dtype: int64

In [117]:
df_skin['skin_type']

0      Da thường/Mọi loại da
1      Da thường/Mọi loại da
2         Da khô/Hỗn hợp khô
3      Da thường/Mọi loại da
4      Da thường/Mọi loại da
               ...          
848       Da dầu/Hỗn hợp dầu
849       Da khô/Hỗn hợp khô
850              Da nhạy cảm
851              Da nhạy cảm
852       Da dầu/Hỗn hợp dầu
Name: skin_type, Length: 853, dtype: object

In [84]:
from gensim.models import FastText

ft_model = FastText(
    sentences=X_skin,
    vector_size=300,
    window=5,
    min_count=2,
    sg=1,          # skip-gram
    epochs=20
)


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


In [85]:
import numpy as np

def sent_vector(tokens, model, default_dim=300):
    vecs = []
    for tok in tokens:
        if tok in model.wv:
            vecs.append(model.wv[tok])
    if not vecs:
        return np.zeros(default_dim)
    return np.mean(vecs, axis=0)

X_res = np.vstack([sent_vector(toks, ft_model) for toks in X_skin])

In [118]:
X_train_skin, X_test_skin, y_train_skin, y_test_skin = train_test_split(
    X_res, df_skin['skin_type'], test_size=0.2, random_state=42, stratify = df_skin['skin_type']
)

In [132]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.svm import SVC, LinearSVR
clf = OneVsRestClassifier(LinearSVC())
clf.fit(X_train_skin, y_train_skin)

,estimator,LinearSVC()
,n_jobs,None
,verbose,0
,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1


In [133]:
Y_pred_skin = clf.predict(X_test_skin)

In [134]:
print("=== SVM Regression ===")
print("Accuracy:", accuracy_score(y_test_skin, Y_pred_skin))

=== SVM Regression ===
Accuracy: 0.7134502923976608


In [135]:
from sklearn.metrics import f1_score, classification_report
print("F1 micro:", f1_score(y_test_skin, Y_pred_skin, average="micro"))
print("F1 macro:", f1_score(y_test_skin, Y_pred_skin, average="macro"))
print(classification_report(y_test_skin, Y_pred_skin))

F1 micro: 0.7134502923976608
F1 macro: 0.5949800909199405
                       precision    recall  f1-score   support

   Da dầu/Hỗn hợp dầu       0.64      0.38      0.47        24
   Da khô/Hỗn hợp khô       0.60      0.18      0.27        17
               Da mụn       0.72      0.75      0.73        24
          Da nhạy cảm       0.86      0.57      0.69        21
Da thường/Mọi loại da       0.71      0.94      0.81        85

             accuracy                           0.71       171
            macro avg       0.71      0.56      0.59       171
         weighted avg       0.71      0.71      0.68       171



In [1]:
from easynmt import EasyNMT
from .autonotebook import tqdm as notebook_tqdm
model = EasyNMT('opus-mt')


/opt/anaconda3/envs/tf310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text = "Xin chào, tôi đang thử EasyNMT."
translated = model.translate(text, target_lang='en')
print(translated)

100%|██████████| 938k/938k [00:00<00:00, 6.97MB/s]
/opt/anaconda3/envs/tf310/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Hi, I'm trying EasyNMT.
